# HW №1
## Task 1

In [1]:
%%capture
!pip3 install qiskit

In [2]:
import numpy as np
import qiskit as qk
from sympy import *

In [3]:
with open('./token', 'r') as token_file:
    token = token_file.read()

In [4]:
qk.IBMQ.save_account(token, overwrite = True)
qk.IBMQ.load_account()

/usr/local/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = qk.IBMQ.get_provider(hub = 'ibm-q')

In [6]:
devices = provider.backends(filters=lambda x: (3 <= x.configuration().n_qubits <= 5) and not x.configuration().simulator)

In [7]:
real_backend = qk.providers.ibmq.least_busy(devices)
print(real_backend.configuration().n_qubits)

TypeError: string indices must be integers

In [ ]:
simd_backend = qk.Aer.get_backend('qasm_simulator')

## Task 2

Since we assume the following form of QBit wavefunction:
$\vert \psi \rangle = \sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle$

Actually we measure kinda projection on $Z$ - axis that corresponds $\theta$ angle. Thus $\phi$ angle describes projection onto $X,~Y$ axes. To fix it we can rotate system (or alternavely Vector) in the following way: $X,~Y,~Z \rightarrow Z,~X,~Y$. To do this we should choose axis of rotation, obviously it is normalized vector $(1,~1,~1)$ and angle to rotate $\alpha = 2 \pi / 3$.

To make rotation itself we should compute corresponding axis. We know the form of such rotations (it can be derived from infinitesimal rotations): $\hat{R}_{\vec{n}} (\alpha)= e^{-i \vec{n} \cdot \hat{\vec{\sigma}} \alpha / 2}$.
Since all sigma's satisfy $\hat{\sigma}_i^2 = \hat{I}$ it can be rewritten in the following form: 

$$\hat{R}_{\vec{n}} (\alpha) = I \cos{\frac{\alpha}{2}} - i \vec{n} \cdot \hat{\vec{\sigma}} \sin{\frac{\alpha}{2}}$$

Thus the function to compute such operator is:

In [ ]:
px = np.array([[0, 1 ], [1 , 0]], dtype = 'complex128')
py = np.array([[0,-1j], [1j, 0]], dtype = 'complex128')
pz = np.array([[1, 0 ], [0 ,-1]], dtype = 'complex128')
pi = np.array([[1, 0 ], [0 , 1]], dtype = 'complex128')

In [ ]:
def rot_mat(vec, angle):
    assert len(vec) == 3
    lvec = np.array(vec, dtype = 'float64')
    lvec = lvec / np.linalg.norm(lvec)
    mat = lvec[0] * px + lvec[1] * py + lvec[2] * pz
    sq_mat = mat @ mat
    assert np.max(np.abs(sq_mat - pi)) < 1e-4
    res = pi * np.cos(angle / 2) - 1j * mat * np.sin(angle / 2)
    sq_res = np.conjugate(res).T @ res
    assert np.max(np.abs(sq_res - pi)) < 1e-4
    return res

We needs not general but specific rotation:

In [ ]:
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)

In [ ]:
print(rot)

We need to choose some random (but normalized) initial state:

In [ ]:
state = np.random.rand(2) + 1j * np.random.rand(2)
state = state / np.linalg.norm(state)
psi_angle = np.angle(state[0])
state = np.exp(-1j * psi_angle) * state
assert np.imag(state[0]) < 1.e-4

Since here state has form: 
$\vert \psi \rangle = e^{i\psi} \sin \theta \vert 0 \rangle + e^{i\phi} \cos \theta \vert 1 \rangle$
We can rewrite in the following form:
$\vert \psi \rangle = e^{i\psi} (\sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle)$, thus $\varphi = \phi - \psi$.

In [ ]:
varphi = np.angle(state[1])
theta  = np.arcsin(state[0])

In [ ]:
varphi, theta

Let's compute symbolic result:

In [ ]:
tht, phi = symbols('theta, phi', real = True)

In [ ]:
vec0 = Matrix([sin(theta), exp(I * phi) * cos(tht)])

In [ ]:
rot_op = Matrix([[1-I, -1-I], [1-I, 1+I]]) / 2

In [ ]:
vec1 = rot_op @ vec0

In [ ]:
ratio = ((conjugate(vec1[0]) * vec1[0]) / (conjugate(vec1[1]) * vec1[1]))

In [ ]:
ratio.simplify()

As we can see ratio of probabilities is:
$$\frac{p(\vert 0 \rangle)}{p(\vert 1 \rangle)} = - \frac{e^{i\phi} (\sin{2\theta} \sin{\phi} + 1)}{e^{i\phi} (\sin{2\theta} \sin{\phi} - 1)} = r$$
Thus the output is:
$$\sin{\phi} \sin{2 \theta} = \frac{r - 1}{r + 1}$$

In [ ]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.initialize(state, q)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 8192).result()
r_simd = res.get_counts()

In [ ]:
print(r_simd)

In [ ]:
qc.draw()

Let's compute $\sin \phi$:

In [ ]:
r = r_simd['0'] / r_simd['1']
sin_phi = (r - 1) / (r + 1) / np.sin(2 * theta)

In [ ]:
res_phi = np.arcsin(sin_phi)

In [ ]:
res_phi

## Task 3

In [ ]:
hm = np.array([[1, 1], [1, -1]], dtype = 'complex128') / np.sqrt(2)
sm = np.array([[1, 0], [0, 1j]], dtype = 'complex128')

We get three angles $\alpha, \beta, \gamma$. Steps of Euler rotation are:

1. Perform $\hat{R}_z (\gamma)$
2. Perform $\hat{R}_x (\beta )$
3. Perform $\hat{R}_z (\gamma)$

To do so we need at least two things:

1. Express $\hat{R}_x$ as combination of $\hat{H},\hat{S},\hat{X}$ and $\hat{R}_z$

## Task 4

### Sub-Task 1

In [8]:
hadm = Matrix([[1, 1], [1, -1]]) / sqrt(2)
xmat = Matrix([[0, 1], [1, 0]])

In [9]:
res1 = hadm @ xmat @ hadm

In [10]:
res1

Matrix([
[1,  0],
[0, -1]])

In [11]:
zmat = Matrix([[1, 0], [0, -1]])

In [12]:
res1 - zmat

Matrix([
[0, 0],
[0, 0]])

### Sub-Task 2

Let's find Controlled Z matrix by definition: it should apply $\hat Z$ to first qubit if the second one (control) is in $\vert 1 \rangle_2$ state. That gives us the following transition table:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|----------------------------|
|$\vert 00 \rangle$  | $+1 \cdot \vert 00 \rangle$|
|$\vert 10 \rangle$  | $+1 \cdot \vert 10 \rangle$|
|$\vert 01 \rangle$  | $+1 \cdot \vert 01 \rangle$|
|$\vert 11 \rangle$  | $-1 \cdot \vert 11 \rangle$|

Thus it can be represented in form of matrix as:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &-1
    \end{bmatrix}    $$
    
Or by formula: $\hat{CZ}: \vert ij \rangle \rightarrow (-1)^{i \cdot j} \vert ij \rangle$, obviously it symmetrical for $i,j$ since that $\hat{CZ}$ is equal in both cases.


### Sub-Task 3

First of all we should determine what is Hadamar operator in terms of 2-qubits. Lets's assume that it is applied to the first one, thus transition table is:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|-----------------------------------------------------------------|
|$\vert 0x \rangle$  | $\frac{1}{\sqrt{2}} \cdot (\vert 0x \rangle + \vert 1x \rangle)$|
|$\vert 1x \rangle$  | $\frac{1}{\sqrt{2}} \cdot (\vert 0x \rangle - \vert 1x \rangle)$|

Or it can be written as a tensor product:
$\hat{H}^{(2)}_{1} = \hat{I}^{(1)}_{2} \otimes \hat{H}^{(1)}_{1}$. That gives us the followinfg form:

In [16]:
from sympy.physics.quantum import TensorProduct

In [17]:
hadm_2_1 = TensorProduct(eye(2), hadm)

In [18]:
hadm_2_1

Matrix([
[sqrt(2)/2,  sqrt(2)/2,         0,          0],
[sqrt(2)/2, -sqrt(2)/2,         0,          0],
[        0,          0, sqrt(2)/2,  sqrt(2)/2],
[        0,          0, sqrt(2)/2, -sqrt(2)/2]])

And vice-versa for hadamar gate on the second qubit:

In [22]:
hadm_2_2 = TensorProduct(hadm, eye(2))

In [23]:
hadm_2_2

Matrix([
[sqrt(2)/2,         0,  sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0,  sqrt(2)/2],
[sqrt(2)/2,         0, -sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0, -sqrt(2)/2]])

The definition of controlled not for the second gate: it should invert the second qubit if the first one is in $\vert 1 \rangle$ state.

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|-------------------|
|$\vert 00 \rangle$  | $\vert 00 \rangle$|
|$\vert 10 \rangle$  | $\vert 11 \rangle$|
|$\vert 01 \rangle$  | $\vert 01 \rangle$|
|$\vert 11 \rangle$  | $\vert 10 \rangle$|

That gives us the following form:

In [32]:
cnot_2 = Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])

In [33]:
cnot_2

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0],
[0, 1, 0, 0]])

Let's writre the right part of circuit:

In [34]:
rc = hadm_2_1 @ hadm_2_2 @ cnot_2 @ hadm_2_1 @ hadm_2_2

In [35]:
rc

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0]])

In [37]:
cnot = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [38]:
rc - cnot

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### Sub-Task 3

We need to find controlled $e^{i\alpha}$ gate. It should work as: multiply both components 
of the second cubit on $e^{i\alpha}$ if the first one is in $\vert 1 \langle$ state. It's diagram:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|--------------------------------------|
|$\vert 00 \rangle$  | $1            \cdot \vert 00 \rangle$|
|$\vert 10 \rangle$  | $e^{i \alpha} \cdot \vert 10 \rangle$|
|$\vert 01 \rangle$  | $1            \cdot \vert 01 \rangle$|
|$\vert 11 \rangle$  | $e^{i \alpha} \cdot \vert 11 \rangle$|

Thus the operator's matrix is the following:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & e^{i\alpha} & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &e^{i\alpha}
    \end{bmatrix}    $$
    
Obviously it's just another representation of the following tensor product:

$\hat{CExp} = I^{(1)}_{2} \otimes \hat{U}_1 (\alpha)$ and that corresponds just $U_1$ gate on the first qubit (see Sub-Task 3).